# Off momentum drift time problem

In this simulation of a propagation through a 1 m drift, particle #2 has a $p_t$ of $10^{-2}$.|

The calculation of $\beta$ based on $p_t$:

$$ p_t = -\frac{\Delta E}{p_0}$$
$$ p_t = - \frac{E - E_0}{p_0} $$
$$ p_t = - \frac{m \gamma - m \gamma_0}{m \beta_0 \gamma_0} $$

Resulting in:
$$ \gamma = - \beta_0 \gamma_0 p_t + \gamma_0 $$

For a proton with kinetic energy of $0.8\,\rm{G}eV$ with $p_t$ of $10^{-2}$ traversing 1 m

In [6]:
from scipy.constants import proton_mass as mp_kg, speed_of_light as c, elementary_charge as qe
from numpy import sqrt

In [14]:
mp_gev = mp_kg * c**2 * 1.0e-9/qe
print('proton mass: ', mp_gev, 'GeV')

proton mass:  0.9382720894282575 GeV


In [17]:
# extract kinematic quantities from saved particle data
import openpmd_api as io
series = io.Series("diags/openPMD/monitor.h5", io.Access.read_only)
beam = series.iterations[1].particles['beam']
gamma0 = beam.get_attribute('gamma_ref')
betagamma0 = beam.get_attribute('beta_gamma_ref')
beta0 = beam.get_attribute('beta_ref')
print('gamma_0: ', gamma0)
print('betagamma_0: ', betagamma0)
print('beta_0: ', beta0)
print('gamma_0**2(1 - beta_0**2) - 1: ', gamma0**2 * (1 - beta0**2) - 1)
print('gamma_0**2 - beta_gamma_0**2 - 1: ', gamma0**2 - betagamma_0**2 - 1)


gamma_0:  1.8526311781434015
betagamma_0:  1.5595647733354994
beta_0:  0.8418107131816728
gamma_0**2(1 - beta_0**2) - 1:  2.220446049250313e-16
gamma_0**2 - beta_gamma_0**2 - 1:  0.0


In [8]:
# Calculate quantities at pt of 1.0e-2
pt = 1.0e-2
gamma = - beta0*gamma0*pt + gamma0
betagamma = sqrt(gamma**2 - 1)
beta = betagamma/gamma
print('gamma: ', gamma)
print('betagamma: ', betagamma)
print('beta: ', beta)

gamma:  1.8370355304100465
betagamma:  1.5410060155589662
beta:  0.838854769028336


Calculate what $t$ should be for a particle

In [9]:
L = 1 # 1 meter
t0 = L/beta0
t1 = L/beta
t = t1 - t0
print('t0 for reference particle: ', t0)
print('t1 for particle with pt=1.0e-2: ', t1)
print('t1 - t0: ', t)

t0 for reference particle:  1.1879155068250933
t1 for particle with pt=1.0e-2:  1.1921014660956415
t1 - t0:  0.004185959270548212


### Get results from simulation

In [10]:
import openpmd_api as io

In [11]:
series = io.Series("diags/openPMD/monitor.h5", io.Access.read_only)
iterations = list(series.iterations)
iter0 = series.iterations[iterations[0]]
iter1 = series.iterations[iterations[-1]]

In [12]:
beam_i = iter0.particles['beam'].to_df() # initial beam
x_i = beam_i['position_x']
y_i = beam_i['position_y']
t_i = beam_i['position_t']
px_i = beam_i['momentum_x']
py_i = beam_i['momentum_y']
pt_i = beam_i['momentum_t']

In [13]:
beam_f = iter1.particles['beam'].to_df()
x_f = beam_f['position_x']
y_f = beam_f['position_y']
t_f = beam_f['position_t']
px_f = beam_f['momentum_x']
py_f = beam_f['momentum_y']
pt_f = beam_f['momentum_t']

In [14]:
print('particle 2 initial coordinates:')
print(x_i[2], y_i[2], t_i[2], px_i[2], py_i[2], pt_i[2])

particle 2 initial coordinates:
0.0 0.0 0.0 0.0 0.0 0.01


In [15]:
print('particle 2 final coordinates:')
print(x_f[2], y_f[2], t_f[2], px_f[2], py_f[2], pt_f[2])

particle 2 final coordinates:
0.0 0.0 0.00418595927054799 0.0 0.0 0.01


In [16]:
# Relative difference between calculation and propagation:
(t1 - t0)/t_f[2] - 1

np.float64(5.306866057708248e-14)